# Init

In [ ]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# from google.colab.patches import cv2_imshow
import sys
# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
import os
import random
import pathlib
import re
import numpy as np
import pandas as pd
# Tensorflow
import tensorflow as tf
from tensorflow import keras
from keras import metrics as metrics
from tensorflow.keras.callbacks import TensorBoard

# XGBoost
from sklearn.metrics import confusion_matrix
import xgboost as xgb
import lightgbm as clf  #faster better

#Augmentations

%matplotlib inline
%load_ext tensorboard

start_data = '/content/drive/MyDrive/ColabNotebooks/Datas/plantvillage-dataset/color/'
dir_path= r'/content/drive/MyDrive/ColabNotebooks/Datas/Plantvilage_that_we_will_use/'
v_path=r'/content/drive/MyDrive/ColabNotebooks/Datas/V_plant/'
t_path=r'/content/drive/MyDrive/ColabNotebooks/Datas/T_plant/'





In [ ]:
# !nvidia-smi
# device_name = tf.test.gpu_device_name()

# print('Found GPU at: {}'.format(device_name))

# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#   print('Not using a high-RAM runtime')
# else:
#   print('You are using a high-RAM runtime!')

# tf.__version__

/bin/bash: line 1: nvidia-smi: command not found
Found GPU at: 
Num GPUs Available:  0
Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


'2.17.1'

In [ ]:
# import tensorflow_datasets as tfds
# ds_test= tfds.load(
#     'mnist',
#     split='train',
#     shuffle_files=True,
#     as_supervised=True,
#     with_info=False
# )

# def normalize_img(image, label):
#   """Normalizes images: `uint8` -> `float32`."""
#   return tf.cast(image, tf.float32) / 255., label

# ds_train = ds_test[0].map(
#     normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# train_data = tf.keras.utils.image_dataset_from_directory(dir_path,
#                                              shuffle=True,
#                                              label_mode = 'categorical',
#                                              batch_size=BATCH_SIZE,
#                                              seed = 42,
#                                              image_size=(IMG_SIZE,IMG_SIZE))

# validation_data = tf.keras.utils.image_dataset_from_directory(v_path,
#                                              shuffle=False,
#                                              label_mode = 'categorical',
#                                              batch_size=BATCH_SIZE,
#                                              image_size=(IMG_SIZE,IMG_SIZE))

# test_data = tf.keras.utils.image_dataset_from_directory(t_path,
#                                              shuffle=False,
#                                              label_mode = 'categorical',
#                                              batch_size=BATCH_SIZE,
#                                              image_size=(IMG_SIZE,IMG_SIZE))

# class_names = train_data.class_names
# num_classes = len(class_names)
# for i in range(1, num_classes + 1):
#     print(str(i) + ". ", class_names[i - 1])


# train = train_data.prefetch(buffer_size=AUTOTUNE)
# val = validation_data.prefetch(buffer_size=AUTOTUNE)
# test=test_data.prefetch(buffer_size=AUTOTUNE)
# print(train)
# print('TRAIN data Batches',len(train))
# print(val)
# print('Val Batches=',len(val))
# print('Test Batches=',len(test_data))

In [ ]:
# tfds.benchmark(train, batch_size=128)

# Build Model class!


TODO
1.   H5 as a hole model will fail.
  
> It is used only on _load_model_from_file and it is used in apply_head_config. So no functionality to load and check if it is a hole model.



2.   Check up for Sequential, Functional API models and check the [-1] layer and update the classes num in the softmax

> Basicaly At the moment we have stored Sequential models with predifined softmax. we dont check the validity.





In [ ]:
from tensorflow.keras.models import model_from_json

""""
A class responsible of loading a number of Complete models or parts of it, from the user if None will produce 1 pre-defined complete model.

None output-->> MobileNetv2 + 256_0.5_128_64_0.5_Num

Model consists of Base/Body Model:
  aka feature extractor
and
  the head aka classifier.

Files that can be loaded are

.h5 which means that will be loaded with weights and model structure aka model config.
  It should not include a softmax  /// SHOULD BE CONFIGURED AND [-1]

.json || .txt files that describe the layers of the classifier
  and the weights of .h5

"""

class Builder:
    def __init__(self, input_shape=256, num_classes=26, head_configs=None, base_models=None, head_weights=None):
        self.input_shape = (input_shape, input_shape, 3)
        self.num_classes = num_classes
        self.head_configs = head_configs if head_configs is not None else []
        self.base_models = base_models if base_models is not None else ["MobileNetV2"]
        self.head_weights = head_weights

# Pre-Defined Base Models
        self.model_dict = {
            "MobileNetV2": {
                "application": tf.keras.applications.MobileNetV2,
                "preprocess": tf.keras.applications.mobilenet_v2.preprocess_input
            },
            "ResNet50": {
                "application": tf.keras.applications.ResNet50,
                "preprocess": tf.keras.applications.resnet50.preprocess_input
            },
            "EfficientNetB0": {
                "application": tf.keras.applications.EfficientNetB0,
                "preprocess": tf.keras.applications.efficientnet.preprocess_input
            },
            "ResNetv2":{
                "application": tf.keras.applications.ResNet50V2,
                "preprocess": tf.keras.applications.resnet_v2.preprocess_input
            }
            # Add more models here as needed
        }
    def _default_head_config(self, x):
      #the Default will be a desnse layer 256 , Drop 0.5, Dense 128, Dense 64, Drop 0.5
        x = tf.keras.layers.Dense(256, activation='relu')(x)
        x = tf.keras.layers.Dropout(rate=0.5)(x)
        x = tf.keras.layers.Dense(128, activation='relu')(x)
        x = tf.keras.layers.Dense(64, activation='relu')(x)
        x = tf.keras.layers.Dropout(rate=0.5)(x)
        x = tf.keras.layers.Dense(self.num_classes, activation='softmax')(x)
        return x

    def _get_base_model(self, base_model_name):
      #Prepares the base models that are freezed and stacked with Preprocessing step, GA Pooling and Flatten layer
        if base_model_name not in self.model_dict:
            raise ValueError(f"Base model {base_model_name} is not supported.")

        # load the the responsible layers from the defined dictionary
        model_config = self.model_dict[base_model_name]
        application = model_config["application"]
        preprocess = model_config["preprocess"]

        base_model = application(input_shape=self.input_shape, include_top=False, weights='imagenet')
        base_model.trainable = False  # Make the base model non-trainable

        inputs = tf.keras.Input(shape=self.input_shape)
        x = preprocess(inputs)
        x = base_model(x, training=False)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Flatten()(x)
        return inputs,x


    def _load_model_from_file(self, file_path):
      #Check the extension type(e.g last split_token_character) and load the file accordingly.
        ext = os.path.splitext(file_path)[-1].lower()
        if ext == '.h5':
            return tf.keras.models.load_model(file_path)
        elif ext in ['.json', '.txt']:
            with open(file_path, 'r') as f:
                config = f.read()
            return model_from_json(config)
        else:
            raise ValueError("Unsupported file type for head configuration.")

    def _apply_head_config(self, x, head_config):
        if isinstance(head_config, str) and os.path.isfile(head_config):
            head_model = self._load_model_from_file(head_config)
            print(head_model.summary())
            head_model.build(input_shape=x.shape)

            #if head_model.layers[-1] isinstance(last_layer, tf.keras.layers.Dense) and head_model.layers[-1].activation == tf.keras.activations.softmax:

            x = head_model(x)
        elif callable(head_config):
            x = head_config(x)
        return x



    def build(self):
        models = []

        if isinstance(self.base_models, str):
            self.base_models = [self.base_models]

        if not isinstance(self.head_configs, list):
            self.head_configs = [self.head_configs]
        # For each model/name split input & output of their base_models. Connect the head
        for base_model_name in self.base_models:
            inputs, base_outputs = self._get_base_model(base_model_name)
            if not self.head_configs:
                # If no head_configs are provided, use the default head configuration
                x = self._default_head_config(base_outputs)
                outputs = x
                model = tf.keras.Model(inputs=inputs, outputs=outputs)
                models.append(model)
            else:
                for head_config in self.head_configs:
                  x = self._apply_head_config(base_outputs, head_config)
                  print(x)
                  #x = tf.keras.layers.Dense(self.num_classes, activation='softmax')(x)
                  outputs = x
                  model = tf.keras.Model(inputs=inputs, outputs=outputs)
                  if self.head_weights:
                      model.load_weights(self.head_weights)
                  models.append(model)

        return models

# '''35'''
# BATCH_SIZE = 128
# base_learning_rate=0.0001
# IMG_SIZE = 256
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# model_list=['/content/drive/MyDrive/ColabNotebooks/Final_thesis_code/Saved_models_and_csv/Defaults/Heads/256_0.5_128_0.5_128_0.5.json','/content/drive/MyDrive/ColabNotebooks/Final_thesis_code/Saved_models_and_csv/Defaults/Heads/256_0.5_128_0.5_64_0.5.json','/content/drive/MyDrive/ColabNotebooks/Final_thesis_code/Saved_models_and_csv/Defaults/Heads/256_0.5_128_128_0.5.json','/content/drive/MyDrive/ColabNotebooks/Final_thesis_code/Saved_models_and_csv/Defaults/Heads/256_0.5_128_64_0.5.json']

# builder = Builder(input_shape=256, num_classes=26, base_models=["MobileNetV2"],head_configs=model_list)
# models = builder.build()

# metrics=[ tf.metrics.CategoricalAccuracy(name="acc"),
#       tf.keras.losses.CategoricalCrossentropy(name='loss'),
#       tf.keras.metrics.F1Score(average="macro", threshold=None, name='f1_score'),
#       tf.metrics.Precision(name='precision'),
#       tf.metrics.Recall(name='recall')]

# # Compile each model

# for model in models:
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
#                     loss=tf.keras.losses.CategoricalCrossentropy(name='loss'),
#                     metrics=metrics)
#     print(model.summary(),sep="\n")


<ipython-input-5-46d3807ca799>:63: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = application(input_shape=self.input_shape, include_top=False, weights='imagenet')


Model: "256_0.5_128_0.5_128_0.5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ true_divide_2 (TrueDivide)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ subtract_2 (Subtract)                │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 8, 8, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ 256_0.5_128_64_0.5 (Sequential)      │ (None, 26)                  │         370,778 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,628,762 (10.03 MB)

 Trainable params: 370,778 (1.41 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

None


In [ ]:
import datetime
# path='/content/drive/MyDrive/ColabNotebooks/Final_thesis_code/app/'

def Call_meback(save_weights=False,save_metrics=True,stop_early=True,path=None,patience=2,file_name=None,name="model",plot=False):
  if file_name is None :
        file_name = "This_is_epoch={epoch:02d}_loss={val_loss:.2f}_f1={val_f1_score:.2f}.keras"
  if path is None: path = "/kaggle/working/store"
  #time
  time = datetime.datetime.now().strftime("_%d-%m-%y->%H:%m")
  #Given or generic name
  model_dir = os.path.join(path,time,name)
  os.makedirs(model_dir, exist_ok=True)
  # END to END filepath
  filep=os.path.join(model_dir,file_name)
  temp=[]



  if save_metrics == True:
    csv=tf.keras.callbacks.CSVLogger(os.path.join(model_dir,'Logs.csv'), separator=",")
    temp.append(csv)
  if stop_early == True:
    stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience,verbose=1)
    temp.append(stop)
  if save_weights == True:
    h5=tf.keras.callbacks.ModelCheckpoint(filepath=filep,
                                          monitor='val_loss',
                                          save_weights_only=False,
                                          save_best_only=True,
                                          initial_value_threshold=0.50,
                                          verbose=1)
    temp.append(h5)
  # elif plot is True: print(ToDo)

  return temp



class MetricsPlotter(tf.keras.callbacks.Callback):
    def __init__(self, metrics=None):
        super().__init__()
        # Use the user’s list, or fall back to these defaults
        self.metrics = metrics or ["loss", "acc","f1_score"]
        # Pre-allocate history lists so we know exactly what keys exist
        self.history = {m: [] for m in self.metrics}
        self.history.update({f"val_{m}": [] for m in self.metrics})

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        # Append whatever values came in (or None if missing)
        for m in self.metrics:
            self.history[m].append(logs.get(m))
            self.history[f"val_{m}"].append(logs.get(f"val_{m}"))

        # Now we can plot in a consistent order:
        n = len(self.metrics)
        clear_output(wait=True)
        plt.figure(figsize=(4 * n, 4))
        for i, m in enumerate(self.metrics, 1):
            plt.subplot(1, n, i)
            plt.plot(self.history[m],   label=m)
            plt.plot(self.history[f"val_{m}"], label=f"val_{m}")
            plt.title(m)
            plt.xlabel("Epoch")
            plt.legend()
        plt.tight_layout()
        plt.show()


NameError: name 'train_data' is not defined

In [ ]:
# list_of_fits=[]
# for i in models:
#   h=i.fit(train_data,
#            epochs=5,
#            validation_data=val,
#            callbacks=Call_meback(save_weights=True,save_metrics=True,stop_early=True,name=i.name))
#   list_of_fits.append(h.history())